In [ ]:
import numpy as np
import collections
from torch.utils import data

from pathlib import Path

import sys
sys.path.append('.')
sys.path.append('../')

from einops import rearrange

import torch
import torch.nn as nn
from torch.nn import functional as FeatureAlphaDropout
import pandas as pd
import matplotlib.pyplot as plt

from torch.utils.data.dataloader import DataLoader

import math
from torch.utils.data import Dataset

from scipy import io as scipyio
import skimage
import skvideo.io

import os
import glob
parent_path = os.path.dirname(os.path.dirname(os.getcwd())) + "/"
sys.path.append("neuroformer")

from analysis import *

In [ ]:
# set up logging
import logging
logging.basicConfig(
        format="%(asctime)s - %(levelname)s - %(name)s -   %(message)s",
        datefmt="%m/%d/%Y %H:%M:%S",
        level=logging.INFO,
)

In [ ]:
from utils import set_seed
set_seed(25)

In [ ]:
# // write a function that find the explicit path of a file sing pathlib
def get_path(path):
    return str(Path(path).resolve())



In [ ]:
# R3D: (3 x T x H x W)

from SpikeVidUtils import image_dataset

stim_folder = "/home/antonis/projects/slab/git/slab/transformer_exp/code/data/OneCombo3/stimuli"
im_path = ['/Combined Stimuli 3-grating.tif',
           '/Combined Stimuli 3-Movie2.tif',
           '/Combined Stimuli 3-Movie3.tif']

train_path = "/content/stimulus"
train_path = "/Users/antonis/Downloads/OneCombo3/stimuli"
video_stack = [skimage.io.imread(stim_folder + vid) for vid in im_path]
print(glob.glob(train_path + '/*.tif'))
video_stack = np.concatenate(video_stack, axis=0)

# video_stack = skimage.io.imread("/home/antonis/projects/slab/git/slab/transformer_exp/code/data/OneCombo3/stimuli/Combined Stimuli 3-grating.tif")
# video_stack = image_dataset(video_stack)
# video_stack = video_stack[::3]  # convert from 60 to 20 fps
# video_stack = video_stack.view(1, video_stack.shape[0], video_stack.shape[1], video_stack.shape[2], video_stack.shape[3])

video_stack = image_dataset(video_stack)
video_stack = video_stack[::3]  # convert from 60 to 20 fps
video_stack = video_stack.view(3, video_stack.shape[0] // 3, video_stack.shape[1], video_stack.shape[2], video_stack.shape[3])
# video_stack = video_stack.transpose(-1, -2)

# rearrange(video_stack[0, 0:2].transpose(0,1), 'c t (h p1) (w p2) -> (t h w) (p1 p2 c)', p1=16, p2=16).shape

In [ ]:
video_stack.shape

In [ ]:
plt.figure()
plt.imshow(video_stack[0, 1, 0].permute(0, 1))
plt.figure()
plt.imshow(video_stack[1, 1, 0].permute(0, 1))
plt.figure()
plt.imshow(video_stack[2, 1, 0].permute(0, 1))

In [ ]:
# spike_path = "/home/antonis/projects/slab/git/slab/transformer_exp/code/data/SImNew3D/neural/NatureMoviePart1-A" # "code/data/SImIm/simNeu_3D_WithNorm__Combo3.mat" 
from SpikeVidUtils import trial_df_combo3

spike_data = scipyio.loadmat("/home/antonis/projects/slab/git/slab/transformer_exp/code/data/OneCombo3/spiketrain.mat")
spike_data = np.squeeze(spike_data['spiketrain'].T, axis=-1)
spike_data = [trial_df_combo3(spike_data, n_stim) for n_stim in range(3)]
spike_data = pd.concat(spike_data, axis=0)

spike_data['Trial'] = spike_data['Trial'] + 1
spike_data['Time'] = spike_data['Time'] * 0.0751
spike_data = spike_data[(spike_data['Time'] > 0) & (spike_data['Time'] <= 32)]

# vid_duration = [len(vid) * 1/20 for vid in vid_list]

df = spike_data
del spike_data

In [ ]:
# df = pd.read_csv(parent_path + "code/data/OneCombo3/Combo3_all_stim.csv")
window = 0.5
dt = 0.05

from SpikeVidUtils import make_intervals

df['Interval'] = make_intervals(df, window)
df['Interval_dt'] = make_intervals(df, dt)
df['Interval_dt'] = (df['Interval_dt'] - df['Interval'] + window).round(3)
df = df.reset_index(drop=True)

In [ ]:
# n_dt = sorted((df['Interval_dt'].unique()).round(2)) 
dt_range = int(window / dt) + 1  # add first / last interval for SOS / EOS'
n_dt = [round(dt * n, 2) for n in range(dt_range)]
df['Time'] = df['Time'].round(3)

In [ ]:
# df.groupby(['Interval', 'Trial']).size().plot.bar()
# df.groupby(['Interval', 'Trial']).agg(['nunique'])
df.groupby(['Interval', 'Trial']).size().nlargest(100)

In [ ]:
from SpikeVidUtils import SpikeTimeVidDataor/stimulus/vq-vae_code_feats-24-05-4x4x4.pt").numpy() + 2
# frame_feats = torch.load(parent_path + "code/data/SImNew3D/stimulus/vq-vae_embed_feats-24-05-4x4x4.pt").numpy()
# frame_block_size = frames.shape[-1] - 1

## resnet3d feats
frame_feats = video_stack.transpose(1, 2)

frame_block_size = int((20 * 64 * 112) / (8 * 8))
# frame_block_size = 560
prev_id_block_size = 52
id_block_size = 52   # 95
block_size = frame_block_size + id_block_size + prev_id_block_size # frame_block_size * 2  # small window for faster training
frame_memory = 20   # how many frames back does model see
window = window

neurons = sorted(list(set(df['ID'])))
id_stoi = { ch:i for i,ch in enumerate(neurons) }
id_itos = { i:ch for i,ch in enumerate(neurons) }

# translate neural embeddings to separate them from ID embeddings
# frames = frames + [*id_stoi.keys()][-1] 
neurons = [i for i in range(df['ID'].min(), df['ID'].max() + 1)]
# pixels = sorted(np.unique(frames).tolist())
feat_encodings = neurons + ['SOS'] + ['EOS'] + ['PAD']  # + pixels 
stoi = { ch:i for i,ch in enumerate(feat_encodings) }
itos = { i:ch for i,ch in enumerate(feat_encodings) }
stoi_dt = { ch:i for i,ch in enumerate(n_dt) }
itos_dt = { i:ch for i,ch in enumerate(n_dt) }
max(list(itos_dt.values()))

In [ ]:
# train_len = round(len(df)*(4/5))
# test_len = round(len(df) - train_len)

# train_data = df[:train_len]
# test_data = df[train_len:train_len + test_len].reset_index().drop(['index'], axis=1)

n = []
for n_stim in range(df['Trial'].max() // 20):
    n_trial = [3, 15, 5, 18]
    for n_trial in n_trial:
        trial = (n_stim + 1) * 20 - n_trial
        n.append(trial)
train_data = df[~df['Trial'].isin(n)].reset_index(drop=True)
test_data = df[df['Trial'].isin(n)].reset_index(drop=True)
small_data = df[df['Trial'].isin([5])].reset_index(drop=True)

In [ ]:
from SpikeVidUtils import SpikeTimeVidData2

# train_dat1aset = spikeTimeData(spikes, block_size, dt, stoi, itos)

train_dataset = SpikeTimeVidData2(train_data, None, block_size, id_block_size, frame_block_size, prev_id_block_size, window, dt, frame_memory, stoi, itos, neurons, stoi_dt, itos_dt, frame_feats, pred=False)
test_dataset = SpikeTimeVidData2(test_data, None, block_size, id_block_size, frame_block_size, prev_id_block_size, window, dt, frame_memory, stoi, itos, neurons, stoi_dt, itos_dt, frame_feats, pred=False)
# dataset = SpikeTimeVidData(df, frames, frame_feats, block_size, frame_block_size, prev_id_block_size, window, frame_memory, stoi, itos)
# single_batch = SpikeTimeVidData(df[df['Trial'].isin([5])], None, block_size, frame_block_size, prev_id_block_size, window, frame_memory, stoi, itos, neurons, stoi_dt, itos_dt, frame_feats)
small_dataset = SpikeTimeVidData2(small_data, None, block_size, id_block_size, frame_block_size, prev_id_block_size, window, dt, frame_memory, stoi, itos, neurons, stoi_dt, itos_dt, frame_feats, pred=False)


print(f'train: {len(train_dataset)}, test: {len(test_dataset)}')

In [ ]:
def get_class_weights(dataset):
  dt = []
  id = []
  for x, y in dataset:
    id.extend([stoi['SOS']] + y['id'][:len(y['id']) - x['pad']].flatten().tolist() + [stoi['PAD']])    # *x['pad']) # -1 in pad to include PAD token
    dt.extend([stoi_dt[0]] + y['dt'][:len(y['dt']) - x['pad']].flatten().tolist() +  [dataset.dt_max])   #*x['pad']) # -1 in pad to include PAD token

  id = pd.DataFrame(id)
  dt = pd.DataFrame(dt)

  id_freq = id.groupby([0]).size()
  dt_freq = dt.groupby([0]).size()

  id_ones = np.ones(dataset.id_population_size)
  dt_ones = np.ones(dataset.dt_population_size)

  id_ones[id_freq.index] = (1 / id_freq) * id_freq.max() / id_freq.max()
  dt_ones[dt_freq.index] = (1 / dt_freq) * dt_freq.max() / dt_freq.max()
  
  class_freq = dict()
  class_freq['id'] = torch.tensor(id_ones, dtype=torch.float32)
  class_freq['dt'] = torch.tensor(dt_ones, dtype=torch.float32)
  
  return class_freq 

class_weights = get_class_weights(train_dataset)

In [ ]:
from model_neuroformer import GPT, GPTConfig, neuralGPTConfig, Decoder
# initialize config class and model (holds hyperparameters)
mconf = GPTConfig(train_dataset.population_size, block_size,    # frame_block_size
                        id_vocab_size=train_dataset.id_population_size,
                        frame_block_size=frame_block_size,
                        id_block_size=id_block_size,  # frame_block_size
                        prev_id_block_size=prev_id_block_size,
                        sparse_mask=False, p_sparse=0.25, sparse_topk_frame=50, sparse_topk_id=id_block_size // 5,
                        n_dt=len(n_dt),
                        data_size=train_dataset.size,
                        class_weights=class_weights,
                        pretrain=False,
                        n_state_layers=4, n_state_history_layers=4, n_stimulus_layers=8,
                        n_layer=10, n_head=4, n_embd=64,
                        temp_emb=True, pos_emb=True,
                        id_drop=0.2, im_drop=0.2)
model = GPT(mconf)
# model.load_state_dict(torch.load("/Users/antonis/Downloads/model_shuffle_perceiver_2.0_dt_0.05_eos_8_8_256.pt", map_location='cpu'))
# model.load_state_dict(torch.load("/home/antonis/projects/slab/git/neuroformer/models/tensorboard/sparse_neuroformer_nopad_unweighted:dt_:True_perceiver_1.0_0.05_(4, 4, 8)_4_64.pt", map_location='cpu'))

In [ ]:
# model.load_state_dict(torch.load("/content/drive/MyDrive/slab/models/OneCombo3/model_8_4_256.pt"))
# model.load_state_dict(torch.load("/content/drive/MyDrive/slab/models/OneCombo3/model_shuffle_8_4_256.pt"))
# model.load_state_dict(torch.load("/content/drive/MyDrive/slab/models/OneCombo3/model_perceiver_81_4_256.pt"))
# model.load_state_dict(torch.load("/content/drive/MyDrive/slab/models/OneCombo3/model_perceiver_83_4_256.pt"))
# model.load_state_dict(torch.load("/content/drive/MyDrive/slab/models/OneCombo3/model_perceiver_dt_8_4_256.pt"))
# model.load_state_dict(torch.load("/content/drive/MyDrive/slab/models/OneCombo3/model_perceiver_dt_8_8_256.pt"))
# model.load_state_dict(torch.load("/content/drive/MyDrive/slab/models/OneCombo3/model_perceiver_dt_eos_8_8_256.pt"))
# model.load_state_dict(torch.load("/content/drive/MyDrive/slab/models/OneCombo3/model_perceiver_2.0_dt_eos_8_8_256.pt"))
# model.load_state_dict(torch.load(f"/content/drive/MyDrive/slab/models/OneCombo3/model_perceiver_2.0_dt:{dt}_eos_{mconf.n_layer}_{mconf.n_head}_{mconf.n_embd}.pt"))
# model.load_state_dict(torch.load(f"/content/drive/MyDrive/slab/models/OneCombo3/model_shuffle_perceiver_2.0_dt:{dt}_eos_{mconf.n_layer}_{mconf.n_head}_{mconf.n_embd}.pt"))
or

In [ ]:
# loader = DataLoader(small_dataset, shuffle=False, pin_memory=False,
#                                   batch_size=1, num_workers=1)
# x, y = next(iter(loader))
# model = model.to('cpu')

# preds, features, loss = model(x, y)

In [ ]:
loader = DataLoader(test_dataset, shuffle=False, pin_memory=False,
                                  batch_size=1, num_workers=1)
                                  

In [ ]:
""" Predict using TEST dataset """

from utils import predict_raster, predict_raster_resnet, predict_raster_enc_dec, predict_raster_recursive, predict_beam_search, predict_raster_recursive_time, predict_raster_recursive_time_auto, predict_beam_search_time, predict_raster_hungarian
from utils import set_plot_params
set_plot_params()
%matplotlib inline

loader = DataLoader(test_dataset, shuffle=False, pin_memory=False,
                                  batch_size=1, num_wororkers=1)

# device = torch.cuda.current_device()
# model = model.to(device)
# model.load_state_dict(torch.load("/Users/antonis/projects/slab/neuroformer/OneCombo3/models/model_weighted_shuffle_nodecay_True_perceiver_2.0_dt_0.05_eos_8_8_256.pt",  map_location='cpu'))

""" 

To predict only neurons we pass <frame_end> so we see predictions only for Neurons 
If you want to also see frame_tokens, just pass <frame_end=0>

NOTE: 512 ID is the <end-of-sequence-id>. Right now, makes no difference if I include
it in loss, here it is included in loss and predictions.

"""

results = predict_raster_recursive_time_auto(model, loader, window, stoi, itos_dt, sample=True, top_p=0.95, top_p_t=0.95, frame_end=0, get_dt=True, gpu=False)

In [ ]:
# whole_dataset = SpikeTimeVidData2(df, None, block_size, id_block_size, frame_block_size, prev_id_block_size, window, dt, frame_memory, stoi, itos, neurons, stoi_dt, itos_dt, frame_feats, pred=False)

# loader = DataLoader(whole_dataset, shuffle=False, pin_memory=False,
#                                   batch_size=1, num_workers=1)

# results = predict_raster_recursive_time_auto(model, loader, window, stoi, itos_dt, sample=True, top_p=0.675, top_p_t=0.675, frame_end=0, get_dt=True, gpu=False)
                                  

In [ ]:
# results = predict_raster_recursive_time_auto(model, loader, window, stoi, itos_dt, sample=True, top_p=0.95, top_p_t=0.95, frame_end=0, get_dt=True, gpu=False)

pred_keys = ['ID', 'dt', 'Trial', 'Interval']
predicted_dict = {k: results[k] for k in results if k in pred_keys}
df_pred = pd.DataFrame(predicted_dict)
df_pred['Time'] = df_pred['dt'] + df_pred['Interval'] - 0.5

In [ ]:
true_keys = ['true', 'time']
true_dict = {k: results[k] for k in results if k in true_keys}
df_true = pd.DataFrame(true_dict)
df_true.rename({'true':'ID', 'time':'dt'}, axis=1, inplace=True)

print(len(test_data), len(df_pred))

In [ ]:
# predicted_timing = [itos_dt[int(dt)] for dt in predicted_timing]
# df_pred = pd.DataFrame({'True':true, 'Predicted':predicted, 'Time':true_timing, 'Predicted_Time':predicted_timing})    # Time':test_data['Time']})
df_pred.to_csv(f"/home/antonis/projects/slab/git/neuroformer/data/SimNeu3D/inference/sparse_neuroformer_nopad_unweighted:dt_:True_perceiver_1.0_0.05_(4, 4, 8)_4_64.csv")

In [ ]:
# df_pred = pd.read_csv("/content/drive/MyDrive/slab/predictions/model_dt_perceiver_83_4_256.csv")

In [ ]:
set_plot_params()

# from utils import set_plot_white
# set_plot_white


In [ ]:
# df_dt = [y['dt'][:len(y['dt']) - (x['pad'])] for x, y in test_dataset]
plt.figure(figsize=(12,10))

# df_dt = df_true['time']
freq_dt_true = df_true.groupby(['dt']).size()
plt.bar(np.arange(len(freq_dt_true.index)), freq_dt_true, alpha=0.5, label='True')


# df_dt_pred = df_pred['time_pred']
freq_dt_pred = df_pred.groupby(['dt']).size()
plt.bar(np.arange(len(freq_dt_pred.index)), freq_dt_pred, alpha=0.5, label='Predicted')


# plt.xticks(ticks=pd.to_numeric(freq_dt_pred.index)labels=pd.to_numeric(freq_dt_pred.index))
plt.title('dt Interval Groups')
plt.xlabel('dt Group (n x 0.05s)')
plt.ylabel('Count (N)')

plt.legend()

plt.tight_layout()
# plt.savefig("dt_interval_dist.png", dpi=300)


In [ ]:
# df_pred = pd.read_csv("OneCombo3/inference/model_weighted_shuffle:True_perceiver_2.0_dt:0.05_eos_8_8_256.csv").iloc[:, 1:]
# # df_pred['Trial'] = test_data['Trial']
# df_pred.rename({'id':'ID', 'trial':'Trial', 'interval':'Interval', 'time':'Time'}, axis=1, inplace=True)
# df_pred.head()


In [ ]:
n_neurons = len(train_data['ID'].unique())

In [ ]:
def get_freqs(df, n_neurons):
    freqs = np.zeros(n_neurons)
    for i in range(n_neurons):
        freqs[i] = (len(df[df['ID'] == i]))
    return freqs

freq_true = get_freqs(test_data, n_neurons)
freq_pred = get_freqs(df_pred[df_pred['ID'] <= max(test_data['ID'].unique())], n_neurons)

plt.xlabel('True Frequency (N)')
plt.ylabel('Predicted Frequency (N)')
plt.xlim(0, max(freq_pred))
plt.scatter(freq_true, freq_pred, alpha=0.5)

In [ ]:
n_trial = [22, 25, 35, 37]
intervals = sorted(set(df['Interval'].unique()) & set(df['Interval'].unique()))
labels = [round(window + window*n, 2) for n in range(0, int(max(df['Interval']) / window))]
ids = sorted(set(df['ID'].unique()) & set(df['ID'].unique()))

In [ ]:
df_1 = test_data[test_data['Trial'].isin(n_trial)]
df_2 = df_pred[df_pred['Trial'].isin(n_trial)]

In [ ]:
df_trials.unique

In [ ]:
from utils import *

set_plot_white()

true_color = "#069AF3"
pred_color = "#F97306"
train_color = "#008000"


df_pred = df_pred[df_pred['ID'] <= max(test_data['ID'].unique())]

n_trials = []
fig, ax = plt.subplots(figsize=(10, 20), nrows=n_trials, ncols=2)

neuron_ids = [i for i in range(20, 26)]
for i in range(neuron_ids):
    chosen_trial = np.random.choice(common_trials)
    freq_true = get_freqs(test_data[test_data['Trial'] == chosen_trial], n_neurons)
    freq_pred = get_freqs(df_pred[df_pred['Trial'] == chosen_trial], n_neurons)
    freq_train = get_freqs(train_data[train_data['Trial'] == chosen_trial + 1], n_neurons)

    ax[i, 0].set_title(f"True vs. Predicted, {chosen_trial}")
    ax[i, 0].scatter(freq_true, freq_pred, alpha=0.5, color=pred_color)
    ax[i, 0].set_xlim(0, max(freq_pred + freq_true))
    ax[i, 0].set_ylim(0, max(freq_pred + freq_true))

    ax[i, 1].set_title(f"True vs. True, {chosen_trial} - {chosen_trial + 1}")
    ax[i, 1].scatter(freq_true, freq_train, alpha=0.5, color=true_color)
    ax[i, 1].set_xlim(0, max(freq_train + freq_true))
    ax[i, 1].set_ylim(0, max(freq_train + freq_true))

    common_trials.remove(chosen_trial)


fig.supxlabel('Count (N)')
fig.supylabel('Count (N)')
fig.suptitle("Trial ID Frequency Comparison", y=0.92, fontsize=16)
plt.tight_layout

In [ ]:
freq_true = test_data.groupby(['ID']).size()
freq_pred = df_pred.groupby(['ID']).size()
plt.scatter(freq_true, freq_pred, alpha=0.5)

In [ ]:
def plot_distribution(df_1, df_2):
    plt.figure(figsize=(30,20))
    n_min = 166
    freq_true = df_1[df_1['ID'] < n_min].groupby(['ID']).size()
    freq_pred = df_2[df_2['ID'] < n_min].groupby(['ID']).size()
    plt.bar(freq_pred.index, freq_pred, label='predicted', alpha=0.5)
    plt.bar(freq_true.index, freq_true, label='true', alpha=0.5)
    plt.title('Neuron Firing Distribution', fontsize=40)
    plt.legend(fontsize=30)
    plt.xlabel('Neuron ID (n)', fontsize=30)
    plt.ylabel('Count (N)', fontsize=30)
    plt.xticks(fontsize=30)
    plt.yticks(fontsize=30)
    plt.savefig("id_interval_dist.png", dpi=300)
    plt.show()


plot_distribution(test_data, df_pred)
# plt.savefig("id_interval_dist.png", dpi=300)

In [ ]:
plot_distribution(df[df['Trial'] == 1], df[df['Trial'] == 2])

In [ ]:
true_timing = np.array(true_timing)
predicted_timing = np.array(predicted_timing)
true_time_seq = [sum(true_timing[0:i[0]]) for i in enumerate(true_timing)]
pred_time_seq = [sum(predicted_timing[0:i[0]]) for i in enumerate(predicted_timing)]

In [ ]:
df_pred

In [ ]:
len_pred = len(df_true)
# len_pred = 1000
plt.figure(figsize=(40,40))
plt.title('Pixel / Spike Raster', size=50)
plt.xlabel('Time')
plt.ylabel('Neuron ID')
plt.scatter(test_data['Time'], test_data['ID'], alpha=0.6, label='true', marker='o')
plt.scatter(df_pred['Time'], df_pred['ID'], alpha=0.6, label='predicted', marker='x')
plt.legend(fontsize=50)

In [ ]:
len_pred = len(true)
# len_pred = 1000
plt.figure(figsize=(40,40))
plt.title('Pixel / Spike Raster', size=50)
plt.xlabel('Time')
plt.ylabel('Neuron ID')
plt.scatter(np.arange(len_pred), true[:len_pred], alpha=0.6, label='true', marker='o')
plt.scatter(np.arange(len_pred), predicted[:len_pred], alpha=0.6, label='predicted', marker='x')
plt.legend(fontsize=50)

In [ ]:
df_pred['Predicted_Time']

In [ ]:
np.arange(len(freq_dt_true.index))

In [ ]:
predicted_timing = [itos_dt for dt in predicted_timing]

In [ ]:
true_timing = pd.DataFrame(true_timing)
predicted_timing = pd.DataFrame(predicted_timing)
plot_distribution(true_timing[true_timing[0] > 0], predicted_timing[predicted_timing[0] > 0])